In [ ]:
import streamlit as st
import pandas as pd
import joblib
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import traceback

st.set_page_config(
    page_title="IFX Drug Concentration Prediction System",
    page_icon="💊",
    layout="wide"
)

# Load pretrained components
@st.cache_resource
def load_components():
    try:
        model = joblib.load(r'C:\Users\cyw\ifx_ensemble_model.pkl')
        scaler = joblib.load(r'C:\Users\cyw\ifx_scaler.pkl')
        return model, scaler
    except FileNotFoundError:
        st.error("Model files not found. Please verify the existence of:\n- ifx_ensemble_model.pkl\n- ifx_scaler.pkl")
        return None, None
    except Exception as e:
        st.error(f"Model loading failed: {str(e)}\n{traceback.format_exc()}")
        return None, None

# Initialize session state
if 'prediction' not in st.session_state:
    st.session_state.prediction = None

# Feature definitions (with complete data types and clinical constraints)
features = {
    'Fg': ('Fibrinogen (g/L)', (1.8, 4.5)),
    'CDAI': ('Crohn\'s Disease Activity Index', (0, 600)),
    'APTT': ('Activated Partial Thromboplastin Time (sec)', 20.0),
    'eGFR': ('Estimated GFR (ml/min/1.73m²)', (30.0, 160.0)),
    'D-Dimer': ('D-Dimer (mg/L)', 0.5),
    'ALB': ('Albumin (g/L)', (35.0, 55.0)),
    'Dose': ('Dosage (mg)', (200.0, 1000.0)),  # Extended dosage range
    'WBC': ('White Blood Cell Count (×10⁹/L)', (4.0, 10.0)),
    'Age': ('Age', (18.0, 80.0)),
    'AST': ('Aspartate Aminotransferase (U/L)', (8.0, 40.0)),
    'ALT': ('Alanine Aminotransferase (U/L)', (7.0, 56.0)),
    'ADA': ('Adenosine Deaminase (U/L)', 4.0),
    'Lesion site': ('Number of Lesion Sites', (1, 5))  # Integer type handling
}

# Sidebar input interface
with st.sidebar:
    st.header("Patient Information Input")
    
    inputs = {}
    for key, (desc, default) in features.items():
        # Handle integer type for lesion sites
        if key == 'Lesion site':
            min_val, max_val = default
            current_val = (min_val + max_val) // 2
            inputs[key] = st.slider(
                label=desc,
                min_value=min_val,
                max_value=max_val,
                value=current_val,
                step=1,
                format="%d",
                key=key
            )
        # Handle range parameters
        elif isinstance(default, tuple):
            min_val, max_val = default
            current_val = (min_val + max_val) / 2
            inputs[key] = st.slider(
                label=desc,
                min_value=float(min_val),
                max_value=float(max_val),
                value=float(current_val),
                key=key
            )
        # Handle single-value parameters
        else:
            inputs[key] = st.number_input(
                label=desc,
                value=float(default),
                min_value=0.0,
                format="%.1f",
                key=key
            )

# Main interface layout
col1, col2 = st.columns([2, 1])

with col1:
    # Input parameters display
    st.subheader("Input Parameters Overview")
    input_df = pd.DataFrame([inputs]).T.reset_index()
    input_df.columns = ['Parameter', 'Input Value']
    st.dataframe(
        input_df, 
        use_container_width=True,
        column_config={
            "Parameter": st.column_config.TextColumn(width="medium"),
            "Input Value": st.column_config.NumberColumn(
                format="%.2f",
                width="small"
            )
        }
    )

with col2:
    # Prediction functionality area
    st.subheader("Prediction Operations")
    if st.button("Start Prediction", use_container_width=True, type="primary"):
        model, scaler = load_components()
        
        if model and scaler:
            try:
                # Ensure feature order consistency
                feature_order = [
                    'Fg', 'CDAI', 'APTT', 'eGFR', 'D-Dimer',
                    'ALB', 'Dose', 'WBC', 'Age', 'AST',
                    'ALT', 'ADA', 'Lesion site'
                ]
                
                # Convert to DataFrame with proper data types
                input_data = pd.DataFrame([inputs])[feature_order].astype(float)
                
                # Normalization
                scaled_data = scaler.transform(input_data)
                
                # Perform prediction
                proba = model.predict_proba(scaled_data)[0]
                prediction = model.predict(scaled_data)[0]
                
                # Explicit probability mapping
                probability = proba[1] if prediction == 1 else proba[0]
                
                # Store results
                st.session_state.prediction = {
                    'class': "Therapeutic Concentration (≥3 μg/ml)" if prediction == 1 else "Suboptimal Concentration (<3 μg/ml)",
                    'probability': probability
                }
                
            except Exception as e:
                st.error(f"Prediction error: {str(e)}\nTraceback:\n{traceback.format_exc()}")

# Display prediction results
if st.session_state.prediction:
    st.markdown("---")
    result = st.session_state.prediction
    
    st.subheader("Prediction Result")
    # Dynamic styling
    if result['class'].startswith("Therapeutic"):
        status_icon = "✅"
        color = "#28a745"
        advice_icon = "🩺"
    else:
        status_icon = "⚠️"
        color = "#dc3545"
        advice_icon = "📉"
    
    st.markdown(f"""
    <div style="border-radius: 10px; padding: 20px; background-color: {color}10; border-left: 5px solid {color}; margin: 20px 0;">
        <h3 style="color:{color}; margin-top:0;">
            {status_icon} {result['class']}
        </h3>
        <p style="font-size: 16px;">Confidence: <strong>{result['probability']*100:.1f}%</strong></p>
    </div>
    """, unsafe_allow_html=True)
    
    # Clinical recommendations card
    st.markdown(f"""
    <div style="border-radius: 10px; padding: 15px; background-color: #f8f9fa; margin: 20px 0;">
        <h4 style="color:#2c3e50; margin-top:0;">{advice_icon} Clinical Recommendations</h4>
        {"""
        ✅ <strong>Therapeutic Guidance:</strong>
        - Maintain current treatment regimen
        - Monitor drug concentration every 8 weeks
        - Regular inflammatory marker monitoring
        
        ⚠️ <strong>Suboptimal Concentration Strategies:</strong>
        - Check medication adherence
        - Consider dose optimization (+10-20%)
        - Test for anti-drug antibodies
        - Shorten monitoring interval (2-4 weeks)
        """ if result['class'].startswith("Therapeutic") else ""}
    </div>
    """, unsafe_allow_html=True)

# Enhanced sidebar instructions
with st.sidebar:
    st.markdown("---")
    st.markdown("""
    **Clinical Protocol**
    
    📌 Parameter Collection Standards:
    1. CDAI: Data from previous week
    2. Biochemical markers: Fasting measurement pre-dose
    3. Lesion sites: Radiological confirmation
    
    ⏳ Best Practices:
    - Initial monitoring 14 days post-dose
    - Re-evaluate after dose adjustment
    - Defer assessment during active infection
    """)

# Runtime instructions
if __name__ == '__main__':
    # For development environment debugging
    import os
    if not os.path.exists(r'C:\Users\cyw\ifx_ensemble_model.pkl'):
        st.warning("Development Note: Model training files required")
